In [3]:
import requests
import pandas as pd
import ta
import time
from datetime import datetime, timedelta

# === CONFIGURATION ===
api_key = 'f5754d3325dc4149ba98663c7dd9821e'
symbol = 'XAU/USD'
interval = '5min'
start_date = '2025-05-01'
end_date = '2025-07-01'
future_horizon = 12

# === Téléchargement d'une page ===
def fetch_data_page(start_date, end_date):
    url = 'https://api.twelvedata.com/time_series'
    params = {
        'symbol': symbol,
        'interval': interval,
        'start_date': start_date,
        'end_date': end_date,
        'apikey': api_key,
        'format': 'JSON',
        'order': 'ASC',
        'timezone': 'UTC',
        'outputsize': 5000
    }
    response = requests.get(url, params=params)
    data = response.json()
    if 'values' in data:
        return pd.DataFrame(data['values'])
    else:
        print("Erreur:", data)
        return None

# === Boucle historique ===
all_data = pd.DataFrame()
current_start = datetime.strptime(start_date, "%Y-%m-%d")
final_end = datetime.strptime(end_date, "%Y-%m-%d")

print("📥 Téléchargement des données en 5 min par batch...")

while current_start < final_end:
    current_end = current_start + timedelta(days=10)
    if current_end > final_end:
        current_end = final_end

    print(f" - {current_start.date()} -> {current_end.date()}")
    df_page = fetch_data_page(current_start.strftime("%Y-%m-%d"), current_end.strftime("%Y-%m-%d"))
    if df_page is not None and not df_page.empty:
        all_data = pd.concat([all_data, df_page])

    current_start = current_end + timedelta(days=1)
    time.sleep(10)

# === Nettoyage ===
if all_data.empty:
    print("❌ ERREUR : aucune donnée téléchargée.")
    exit()

print("✅ Données téléchargées. Nettoyage...")

all_data['datetime'] = pd.to_datetime(all_data['datetime'])
all_data = all_data.sort_values('datetime').reset_index(drop=True)

for col in ['open', 'high', 'low', 'close']:
    all_data[col] = all_data[col].astype(float)

# === Indicateurs techniques ===
print("✅ Calcul des indicateurs techniques...")

close_series = all_data['close']
high_series = all_data['high']
low_series = all_data['low']

all_data['rsi'] = ta.momentum.RSIIndicator(close_series, window=10).rsi()
all_data['ema_9'] = close_series.ewm(span=9, adjust=False).mean()
all_data['ema_21'] = close_series.ewm(span=21, adjust=False).mean()
all_data['ema_distance'] = abs(all_data['ema_9'] - all_data['ema_21'])

macd_calc = ta.trend.MACD(close_series)
all_data['macd_line'] = macd_calc.macd()

atr_indicator = ta.volatility.AverageTrueRange(high_series, low_series, close_series, window=14)
all_data['atr'] = atr_indicator.average_true_range()

all_data['volatility_close_std'] = close_series.rolling(window=10).std()
all_data['ema_9_slope'] = all_data['ema_9'].diff(3)
all_data['ema_21_slope'] = all_data['ema_21'].diff(3)

# ✅ Nouvelle target : prédire la CLOSE dans 12 heures
print(f"✅ Calcul de la target future_close à {future_horizon} bougies...")
all_data['future_close'] = all_data['close'].shift(-future_horizon)

# Nettoyage final
all_data = all_data.dropna()

print(all_data[['datetime', 'close', 'future_close']].head())

# ✅ Sauvegarde CSV
all_data.to_csv('gold_features_hourly_future_close.csv', index=False)
print("✅ Données sauvegardées dans gold_features_hourly_future_close.csv")


📥 Téléchargement des données en 5 min par batch...
 - 2025-05-01 -> 2025-05-11
 - 2025-05-12 -> 2025-05-22
 - 2025-05-23 -> 2025-06-02
 - 2025-06-03 -> 2025-06-13
 - 2025-06-14 -> 2025-06-24
 - 2025-06-25 -> 2025-07-01
✅ Données téléchargées. Nettoyage...
✅ Calcul des indicateurs techniques...
✅ Calcul de la target future_close à 12 bougies...
              datetime    close  future_close
25 2025-05-01 02:05:00  3239.30       3237.26
26 2025-05-01 02:10:00  3240.29       3231.90
27 2025-05-01 02:15:00  3245.06       3235.39
28 2025-05-01 02:20:00  3250.03       3232.74
29 2025-05-01 02:25:00  3246.39       3233.19
✅ Données sauvegardées dans gold_features_hourly_future_close.csv


In [4]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# === 1️⃣ Chargement des données ===
df = pd.read_csv('gold_features_hourly_future_close.csv')
print("✅ Données chargées")
print(df.head())

# === 2️⃣ Sélection des features ===
features = [
    'rsi',
    'ema_9', 'ema_21', 'ema_distance',
    'macd_line',
    'atr',
    'volatility_close_std',
    'ema_9_slope', 'ema_21_slope'
]

X = df[features]
y = df['future_close']

# === 3️⃣ Split ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)
print("✅ Split terminé")

# === 4️⃣ Entraînement ===
model = RandomForestRegressor(
    n_estimators=200,
    max_depth=12,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)
print("✅ Modèle entraîné")

# === 5️⃣ Évaluation ===
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n✅ Metrics:")
print(f" - MSE : {mse:.4f}")
print(f" - RMSE: {rmse:.4f}")
print(f" - R2  : {r2:.4f}")

# === 6️⃣ Sauvegarde du modèle
with open('gold_future_close_regressor.pkl', 'wb') as f:
    pickle.dump(model, f)
print("✅ Modèle sauvegardé sous gold_future_close_regressor.pkl")


✅ Données chargées
              datetime     open     high      low    close        rsi  \
0  2025-05-01 02:05:00  3240.60  3242.61  3239.08  3239.30  36.037100   
1  2025-05-01 02:10:00  3239.54  3240.29  3235.00  3240.29  38.445588   
2  2025-05-01 02:15:00  3240.17  3246.89  3240.17  3245.06  48.772296   
3  2025-05-01 02:20:00  3244.70  3250.03  3241.19  3250.03  57.103697   
4  2025-05-01 02:25:00  3250.05  3251.29  3245.00  3246.39  50.429483   

         ema_9       ema_21  ema_distance  macd_line       atr  \
0  3241.892523  3247.294002      5.401479  -6.370322  6.712800   
1  3241.572019  3246.657275      5.085256  -6.124599  6.611171   
2  3242.269615  3246.512068      4.242453  -5.481772  6.618945   
3  3243.821692  3246.831880      3.010188  -4.519195  6.777591   
4  3244.335354  3246.791709      2.456356  -4.003909  6.742763   

   volatility_close_std  ema_9_slope  ema_21_slope  future_close  
0              4.282865    -2.244749     -2.573215       3237.26  
1          